## Helpful Documentation:
- https://github.com/kpu/kenlm
- Old but useful: http://victor.chahuneau.fr/notes/2012/07/03/kenlm.html
- http://smithamilli.com/blog/kneser-ney/
- There are more notes here: https://kheafield.com/code/kenlm/estimation/

In [1]:
from article_process import ArticleLM

In [2]:
path_to_data = '/Users/stephanie/data/newsela_article_corpus_2016-01-29'
path_to_kenlm = '/Users/stephanie/github/kenlm'
path_to_arpa = path_to_kenlm + '/lm'

In [3]:
articleLM = ArticleLM(path_to_data, path_to_kenlm, path_to_arpa)

In [4]:
articleLM.metadata_split.head()

,Unnamed: 0,slug,language,title,grade_level,version,filename,train_val_test
0,0,10dollarbill-woman,en,"Tubman, Perkins or Roosevelt? Woman on $10 bil...",12,0,10dollarbill-woman.en.0.txt,test
1,1,10dollarbill-woman,en,Americans weigh in to choose the woman who wil...,8,1,10dollarbill-woman.en.1.txt,train
2,2,10dollarbill-woman,en,The $10 question: Who will be the new face on ...,6,2,10dollarbill-woman.en.2.txt,val
3,3,10dollarbill-woman,en,New $10 bill will have a theme and a woman's p...,5,3,10dollarbill-woman.en.3.txt,train
4,4,10dollarbill-woman,en,We will soon have an American woman's face on ...,3,4,10dollarbill-woman.en.4.txt,train


## Builds the processed sentences that I need for KenLM

In [5]:
articleLM.build_data()

In [6]:
articleLM.grade_level_sentences['train'].keys()

dict_keys([8, 5, 3, 12, 9, 7, 4, 6, 2, 10, 11])

## This will do the training, but I don't train again after I've already done it once.

In [ ]:
# articleML.train_all_arpas()

In [7]:
# Can also train just one grade level at a time
# articleLM.train_arpa(2)

Grade 2 has 6595 sentences in the training set.
=== 1/5 Counting and sorting n-grams ===
Reading stdin
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
****************************************************************************************************
Unigram tokens 61280 types 4878
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:58536 2:1340862080 3:2514116608 4:4022586368 5:5866272256
Statistics:
1 4878 D1=0.563553 D2=1.08447 D3+=1.65137
2 28641 D1=0.773578 D2=1.16304 D3+=1.49466
3 46098 D1=0.880812 D2=1.29085 D3+=1.49004
4 50115 D1=0.943422 D2=1.46295 D3+=1.58142
5 46775 D1=0.971358 D2=1.56956 D3+=1.56239
Memory estimate for binary LM:
type      kB
probing 3872 assuming -p 1.5
probing 4622 assuming -r models -p 1.5
trie    1767 without quantization
trie     927 assuming -q 8 -b 8 quantization 
trie    1632 assuming -a 22 array poi

## Printing out stats

how many sentences there are in training set, indexed by grade level

In [7]:
for g in articleLM.grade_level_sentences.get('train').keys():
    print(g, len(articleLM.grade_level_sentences.get('train').get(g)))

8 35962
5 50250
3 15314
12 62762
9 22757
7 42521
4 59169
6 39236
2 6595
10 791
11 53


how many articles there are in each set, indexed by grade level

In [10]:
articleLM.metadata_split[articleLM.metadata_split.train_val_test=='train'].grade_level.value_counts()

12    1459
4     1208
5     1044
7      945
6      828
8      822
9      594
3      434
2      198
10      16
11       2
Name: grade_level, dtype: int64

In [11]:
articleLM.metadata_split[articleLM.metadata_split.train_val_test=='val'].grade_level.value_counts()

12    306
4     272
7     219
5     205
6     181
8     162
9     132
3     103
2      36
10      2
Name: grade_level, dtype: int64

In [12]:
articleLM.metadata_split[articleLM.metadata_split.train_val_test=='test'].grade_level.value_counts()

12    331
4     250
5     239
7     201
8     168
6     162
9     136
3      79
2      49
10      3
Name: grade_level, dtype: int64

In [ ]:
articleLM.compute_perplexity_validation('val', 2)

In [8]:
def compute_perplexity_validation(split_type, grade_level):
    # Split type can be 'val' or 'test'
    sample_perp = dict()
    for ix, sentence in enumerate(articleLM.grade_level_sentences[split_type][grade_level]):
        perplexities = articleLM.compute_sentence_perplexities(sentence)
        sample_perp[ix] = perplexities
    sample_perp_df = pd.DataFrame(sample_perp).T
    sample_perp_df.columns = sorted(articleLM.models.keys())
    return sample_perp_df

In [10]:
import pandas as pd

In [29]:
for ix, sentence in enumerate(articleLM.grade_level_sentences['val'][2]):
     perplexities = articleLM.compute_sentence_perplexities(sentence)

In [30]:
perplexities

[250.0085957692233,
 326.5919789254105,
 157.64826777289863,
 148.76504699556824,
 160.11189992044885,
 18.014581783703907,
 136.46682216940155,
 17.06018638906965,
 132.41926609524762,
 17.046273216423238]

## Example: This all perplexities comparing each grade level (columns) with each sentence in all of 2nd grade 
Rows indexes the sentence number from articleLM.grade_level_sentences['val'][2]

`articleLM.grade_level_sentences` is a dict that has `train`, `val`, and `test` keys. Each dict has a dict to each grade level, which stores a list of sentences parsed to be pushed into KenLM.

In [31]:
articleLM.compute_perplexity_validation('val', 2)

,2,3,4,5,6,7,8,9,10,12
0,84.424339,138.926475,97.415321,160.666621,34.068183,30.530879,39.798165,46.038845,277.760292,36.331528
1,202.784336,229.388204,282.448552,225.194145,348.317142,105.687024,427.936215,88.025925,288.478780,147.010143
2,138.903380,105.072679,105.491025,133.986747,182.714880,215.837542,234.436165,224.541361,391.991001,260.453799
3,457.518496,252.215614,540.872570,410.795457,191.569320,269.708241,360.872302,320.991448,315.959837,312.953029
4,31.141015,39.184910,39.160197,53.409839,61.578282,40.562555,35.595761,77.872638,121.495877,55.509219
5,173.986850,345.422847,765.347102,826.996355,1307.162770,77.579381,1249.850374,74.933557,405.498333,88.060387
6,45.591640,19.489179,48.304161,23.365376,29.026766,45.103627,114.181009,105.803930,171.139445,36.403055
7,108.469006,137.719785,95.235756,141.340577,202.868358,123.484394,88.911269,170.957066,368.405890,146.920811
8,31.178313,85.608052,37.404702,123.433125,104.621744,215.399633,80.128102,130.020099,284.154803,125.321027
9,42.388803,31.244097,48.399336,48.892943,54.226722,60.831965,65.743716,83.521642,87.469000,93.788110


In [9]:
articleLM.grade_level_sentences['val'].get(6)[5]

'an abolitionist who helped to end slavery .'

In [10]:
articleLM.grade_level_sentences['val'].get(2)[5]

'his name is fazle hasan abed .'

In [29]:
sorted(model.keys())

[2, 3, 4, 5, 6, 7, 8, 9, 10, 12]

In [12]:
# articleLM.text_sentences_for_grade(2, True)

## ARCHIVED

In [17]:
articleLM.train_arpa(2)

Grade 2 has 8140 sentences in the training set.
=== 1/5 Counting and sorting n-grams ===
Reading stdin
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
****************************************************************************************************
Unigram tokens 77147 types 7247
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:86964 2:1340859264 3:2514111488 4:4022577920 5:5866259968
Statistics:
1 7247 D1=0.587801 D2=1.08954 D3+=1.56537
2 37930 D1=0.778791 D2=1.19549 D3+=1.51871
3 59374 D1=0.882787 D2=1.31476 D3+=1.5341
4 63672 D1=0.946344 D2=1.48411 D3+=1.50262
5 59302 D1=0.972426 D2=1.58325 D3+=1.31886
Memory estimate for binary LM:
type      kB
probing 4999 assuming -p 1.5
probing 5970 assuming -r models -p 1.5
trie    2296 without quantization
trie    1213 assuming -q 8 -b 8 quantization 
trie    2121 assuming -a 22 array poin

In [23]:
for grade_level in articleLM.grade_level_train_sentences.keys():
    print('Processing {}'.format(grade_level))
    articleLM.train_arpa(grade_level)

Processing 12
Grade 12 has 70339 sentences in the training set.
=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 1701343 types 67748
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:812976 2:1340788480 3:2513978624 4:4022365440 5:5865950208
Statistics:
1 67748 D1=0.596335 D2=1.0247 D3+=1.51618
2 603184 D1=0.766528 D2=1.13251 D3+=1.44168
3 1226628 D1=0.885567 D2=1.2354 D3+=1.45629
4 1481759 D1=0.953303 D2=1.37475 D3+=1.53093
5 1513009 D1=0.977679 D2=1.43837 D3+=1.48664
Memory estimate for binary LM:
type     MB
probing 103 assuming -p 1.5
probing 122 assuming -r models -p 1.5
trie     50 without quantization
trie     27 assuming -q 8 -b 8 quantization 
trie     44 assuming -a 22 array pointer compression
trie     21 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:812976 2:9650944 3:2453256

=== 1/5 Counting and sorting n-grams ===
Reading stdin
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
****************************************************************************************************
Unigram tokens 198792 types 12972
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:155664 2:1340852608 3:2514098944 4:4022557952 5:5866230784
Statistics:
1 12972 D1=0.583245 D2=1.04729 D3+=1.59745
2 84402 D1=0.76402 D2=1.16715 D3+=1.46525
3 145230 D1=0.877772 D2=1.25399 D3+=1.51978
4 162868 D1=0.941427 D2=1.40244 D3+=1.45998
5 156883 D1=0.967368 D2=1.51688 D3+=1.41989
Memory estimate for binary LM:
type       kB
probing 12286 assuming -p 1.5
probing 14636 assuming -r models -p 1.5
trie     5717 without quantization
trie     3032 assuming -q 8 -b 8 quantization 
trie     5198 assuming -a 22 array pointer compression
trie     2513 assum

In [ ]:
articleLM.train_all_arpas()

In [17]:
import subprocess

In [ ]:
from subprocess import Popen, PIPE, STDOUT

p = Popen(['grep', 'f'], stdout=PIPE, stdin=PIPE, stderr=STDOUT)    
grep_stdout = p.communicate(input=b'one\ntwo\nthree\nfour\nfive\nsix\n')[0]
print(grep_stdout.decode())

NameError: name 'path_to_arpa' is not defined

In [1]:
import nltk
from article import Library
# path = '/Users/stzeng/code/github/autograder/data/newsela_article_corpus_2016-01-29/'
path = '/Users/stephanie/data/newsela_article_corpus_2016-01-29/'
library = Library(path)
library.create_grade_level_library()
# print(library.library['zuckerberg-internet'].article_text(4.0))

library.metadata.groupby('grade_level').count()

,slug,language,title,version,filename
grade_level,,,,,
2,283,283,283,283,283
3,616,616,616,616,616
4,1730,1730,1730,1730,1730
5,1488,1488,1488,1488,1488
6,1171,1171,1171,1171,1171
7,1365,1365,1365,1365,1365
8,1152,1152,1152,1152,1152
9,862,862,862,862,862
10,21,21,21,21,21


In [ ]:
# nltk.download('punkt')

In [2]:
from process import *

In [4]:
path = '/Users/stephanie/data/newsela_article_corpus_2016-01-29/'
metadata = pd.read_csv(path + 'articles_metadata.csv')
metadata.loc[:, 'grade_level'] = metadata.grade_level.astype('int')
metadata_train = pd.read_csv(path + 'articles_metadata_train.csv')
metadata_test = pd.read_csv(path + 'articles_metadata_test.csv')

for gl in [2, 3, 4, 5, 6, 7, 8, 9, 12]:
    print(gl)
    print_grade_level_shapes(metadata_train, metadata_test, gl)

2
Train shape (198, 7)
Test shape (85, 7)
3
Train shape (434, 7)
Test shape (182, 7)
4
Train shape (1208, 7)
Test shape (522, 7)
5
Train shape (1044, 7)
Test shape (444, 7)
6
Train shape (828, 7)
Test shape (343, 7)
7
Train shape (945, 7)
Test shape (420, 7)
8
Train shape (822, 7)
Test shape (330, 7)
9
Train shape (594, 7)
Test shape (268, 7)
12
Train shape (1459, 7)
Test shape (637, 7)


In [5]:
! python process.py 2 | ~/github/kenlm/build/bin/lmplz -o 3 > GL2.arpa

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 77147 types 7247
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:86964 2:4780455424 3:8963353600
Statistics:
1 7247 D1=0.587801 D2=1.08954 D3+=1.56537
2 37930 D1=0.778791 D2=1.19549 D3+=1.51871
3 59374 D1=0.860848 D2=1.28242 D3+=1.40264
Memory estimate for binary LM:
type      kB
probing 2116 assuming -p 1.5
probing 2367 assuming -r models -p 1.5
trie     914 without quantization
trie     533 assuming -q 8 -b 8 quantization 
trie     875 assuming -a 22 array pointer compression
trie     494 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:86964 2:606880 3:1187480
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
###############################################################################

In [6]:
! python process.py 3 | ~/github/kenlm/build/bin/lmplz -o 3 > GL3.arpa

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 198792 types 12972
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:155664 2:4780431360 3:8963308544
Statistics:
1 12972 D1=0.583245 D2=1.04729 D3+=1.59745
2 84402 D1=0.76402 D2=1.16715 D3+=1.46525
3 145230 D1=0.855721 D2=1.20474 D3+=1.45815
Memory estimate for binary LM:
type      kB
probing 4860 assuming -p 1.5
probing 5405 assuming -r models -p 1.5
trie    2080 without quantization
trie    1191 assuming -q 8 -b 8 quantization 
trie    1974 assuming -a 22 array pointer compression
trie    1085 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:155664 2:1350432 3:2904600
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
#########################################################################

In [7]:
! python process.py 4 | ~/github/kenlm/build/bin/lmplz -o 3 > GL4.arpa

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 829750 types 30239
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:362868 2:4780359168 3:8963173376
Statistics:
1 30239 D1=0.577754 D2=1.05543 D3+=1.57045
2 272851 D1=0.753521 D2=1.13059 D3+=1.43538
3 551377 D1=0.846526 D2=1.21969 D3+=1.42214
Memory estimate for binary LM:
type       kB
probing 16854 assuming -p 1.5
probing 18571 assuming -r models -p 1.5
trie     7068 without quantization
trie     3958 assuming -q 8 -b 8 quantization 
trie     6669 assuming -a 22 array pointer compression
trie     3559 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:362868 2:4365616 3:11027540
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
###############################################################

In [8]:
! python process.py 5 | ~/github/kenlm/build/bin/lmplz -o 3 > GL5.arpa

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 819274 types 32541
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:390492 2:4780349440 3:8963154944
Statistics:
1 32541 D1=0.583686 D2=1.08003 D3+=1.54791
2 286102 D1=0.756318 D2=1.14027 D3+=1.4059
3 569252 D1=0.856522 D2=1.21962 D3+=1.40494
Memory estimate for binary LM:
type       kB
probing 17538 assuming -p 1.5
probing 19341 assuming -r models -p 1.5
trie     7381 without quantization
trie     4145 assuming -q 8 -b 8 quantization 
trie     6962 assuming -a 22 array pointer compression
trie     3725 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:390492 2:4577632 3:11385040
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
################################################################

In [9]:
! python process.py 6 | ~/github/kenlm/build/bin/lmplz -o 3 > GL6.arpa

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 733243 types 34508
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:414096 2:4780341248 3:8963139584
Statistics:
1 34508 D1=0.587213 D2=1.05437 D3+=1.5943
2 281994 D1=0.766817 D2=1.14875 D3+=1.41288
3 537093 D1=0.868 D2=1.2397 D3+=1.4185
Memory estimate for binary LM:
type       kB
probing 16926 assuming -p 1.5
probing 18713 assuming -r models -p 1.5
trie     7298 without quantization
trie     4175 assuming -q 8 -b 8 quantization 
trie     6881 assuming -a 22 array pointer compression
trie     3759 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:414096 2:4511904 3:10741860
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
#####################################################################

In [10]:
! python process.py 7 | ~/github/kenlm/build/bin/lmplz -o 3 > GL7.arpa

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 893174 types 40615
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:487380 2:4780316160 3:8963092480
Statistics:
1 40615 D1=0.583532 D2=1.09183 D3+=1.58595
2 339700 D1=0.766086 D2=1.15023 D3+=1.37507
3 655627 D1=0.868356 D2=1.23355 D3+=1.41805
Memory estimate for binary LM:
type       kB
probing 20517 assuming -p 1.5
probing 22666 assuming -r models -p 1.5
trie     8818 without quantization
trie     5032 assuming -q 8 -b 8 quantization 
trie     8318 assuming -a 22 array pointer compression
trie     4531 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:487380 2:5435200 3:13112540
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
###############################################################

In [11]:
! python process.py 8 | ~/github/kenlm/build/bin/lmplz -o 3 > GL8.arpa

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 831418 types 41552
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:498624 2:4780312064 3:8963085312
Statistics:
1 41552 D1=0.59951 D2=1.05162 D3+=1.45627
2 330946 D1=0.770558 D2=1.14571 D3+=1.45273
3 625081 D1=0.874872 D2=1.25378 D3+=1.41979
Memory estimate for binary LM:
type       kB
probing 19799 assuming -p 1.5
probing 21900 assuming -r models -p 1.5
trie     8559 without quantization
trie     4908 assuming -q 8 -b 8 quantization 
trie     8070 assuming -a 22 array pointer compression
trie     4419 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:498624 2:5295136 3:12501620
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
################################################################

In [12]:
! python process.py 9 | ~/github/kenlm/build/bin/lmplz -o 3 > GL9.arpa

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 586228 types 35647
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:427764 2:4780336640 3:8963131392
Statistics:
1 35647 D1=0.594607 D2=1.07896 D3+=1.55584
2 256435 D1=0.778361 D2=1.14546 D3+=1.45445
3 458772 D1=0.882882 D2=1.24915 D3+=1.44144
Memory estimate for binary LM:
type       kB
probing 14979 assuming -p 1.5
probing 16621 assuming -r models -p 1.5
trie     6535 without quantization
trie     3779 assuming -q 8 -b 8 quantization 
trie     6184 assuming -a 22 array pointer compression
trie     3428 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:427764 2:4102960 3:9175440
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
################################################################

In [13]:
! python process.py 12 | ~/github/kenlm/build/bin/lmplz -o 3 > GL12.arpa

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 1701343 types 67748
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:812976 2:4780202496 3:8962880512
Statistics:
1 67748 D1=0.596335 D2=1.0247 D3+=1.51618
2 603184 D1=0.766528 D2=1.13251 D3+=1.44168
3 1226628 D1=0.870462 D2=1.21493 D3+=1.40635
Memory estimate for binary LM:
type       kB
probing 37419 assuming -p 1.5
probing 41218 assuming -r models -p 1.5
trie    16211 without quantization
trie     9310 assuming -q 8 -b 8 quantization 
trie    15255 assuming -a 22 array pointer compression
trie     8354 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:812976 2:9650944 3:24532560
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
##############################################################

In [8]:
import subprocess

In [25]:
bashCommand = "python process.py {gl} | ~/github/kenlm/build/bin/lmplz -o 3 > GL_{gl}.arpa"

In [39]:
for gl in [2, 3, 4, 5, 6, 7, 8, 9, 12]:
    cmd = bashCommand.format(gl=gl)
#     print(cmd.split())
    process = subprocess.Popen(["python",
                                "process.py", str(gl),
                                "|","~/github/kenlm/build/bin/lmplz",
                                "-o", str(gl),
                                ">",
                                "GL_{gl}.arpa".format(gl=gl)
                               ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = process.communicate()

In [40]:
error

b'usage: process.py [-h] grade_level\nprocess.py: error: unrecognized arguments: | ~/github/kenlm/build/bin/lmplz -o 3 > GL_12.arpa\n'